In [1]:
!pip install -r requirements.txt

In [4]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

# Get SageMaker execution role
role = sagemaker.get_execution_role()

# Define the HuggingFace Model Hub configuration
hub = {
    'HF_MODEL_ID': 'google/flan-t5-small',
    'HF_TASK': 'text2text-generation'
}

# Create HuggingFace Model object
huggingface_model = HuggingFaceModel(
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version='py39',
    env=hub,
    role=role
)

# Deploy the model to a SageMaker endpoint (Free Tier eligible instance)
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium'  # Free Tier eligible
)

print('SageMaker endpoint deployed! Endpoint name:', predictor.endpoint_name) 

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
-------------!SageMaker endpoint deployed! Endpoint name: huggingface-pytorch-inference-2025-07-24-16-48-17-100


In [11]:
import boto3
import json

# Replace with your actual SageMaker endpoint name and region
SAGEMAKER_ENDPOINT = 'huggingface-pytorch-inference-2025-07-24-16-48-17-100'
REGION = 'us-east-1'

# Test prompts
test_prompts = [
    "What a piece of crap. Only an idiot would write such garbage code.",
    "Your code is awesome",
    "Wow, did you even try to make this work, or was it just for fun?",
    "I think this code could be improved by adding more comments."
]

# Create SageMaker runtime client
sm_runtime = boto3.client('sagemaker-runtime', region_name=REGION)

for prompt in test_prompts:
    # You may need to format the prompt as your model expects (with instructions/examples)
    payload = {
        "inputs": f"You are an assistant that rewrites software/code-related comments to be neutral, professional, and respectful. "
        "Remove any insults, offensive language, or toxicity, but keep all technical meaning and intent. "
        "Do not add unrelated information or change the topic. Only rephrase the original comment in a more respectful way.\n"
        "\n"
        "Example 1:\n"
        "Original: What a piece of crap. Only an idiot would write such garbage code.\n"
        "Rewritten: This code could be improved. There are several issues that need to be addressed.\n"
        "\n"
        "Example 2:\n"
        "Original: You must be the dumbest person to think this library is useful. Just quit coding forever.\n"
        "Rewritten: I don't find this library useful for my needs. Perhaps consider a different approach.\n"
        "\n"
        "Example 3:\n"
        "Original: This is the worst implementation I've ever seen. Did you even test it?\n"
        "Rewritten: This implementation could be improved. Have you considered testing it further?\n"
        "\n"
        f"Original: {prompt}\nRewritten:"
    }
    response = sm_runtime.invoke_endpoint(
        EndpointName=SAGEMAKER_ENDPOINT,
        ContentType='application/json',
        Body=json.dumps(payload)
    )
    result = json.loads(response['Body'].read())
    rewritten = result[0]['generated_text'] if isinstance(result, list) else result
    print(f"Original: {prompt}")
    print(f"Rewritten: {rewritten}\n{'-'*60}\n")

Original: What a piece of crap. Only an idiot would write such garbage code.
Rewritten: This code could be improved. There are several issues that need to be addressed. Example 1:
------------------------------------------------------------

Original: Your code is awesome
Rewritten: I'm a big fan of the coding.
------------------------------------------------------------

Original: Wow, did you even try to make this work, or was it just for fun?
Rewritten: I'm sorry, but I'm not sure what to do.
------------------------------------------------------------

Original: I think this code could be improved by adding more comments.
Rewritten: I think this code could be improved. Have you considered testing it further?
------------------------------------------------------------

